In [17]:
import pandas as pd
import numpy as np
from pathlib import Path

import module
import importlib
try:
    importlib.reload(module) # reload module
except NameError:
    pass

# Load DF

In [18]:
date= '2023-12-14'# to get csv

In [19]:
path= Path(f'input/{date}.csv')
df= pd.read_csv(path)

target_cols = ["External ID","Display Name","Date of Birth","Email","Phone","Contract Type","From","To","Fully Paid Date","Membership/Display Name","Membership Status","Partner/Branch/Display Name","Partner/Date of Birth","Partner/Age","Partner/City","Partner/Gender","Partner/Household income/month","Partner/Industry/Display Name","Partner/Job Position","Partner/Occupation","Partner/Street","Partner/Street2","Followers (Partners)/Hobby","Followers (Partners)/Interest","Partner/Hobby","Partner/Interest"
]
current_cols = list(df.columns)
map_cols = dict(zip(current_cols,target_cols))
df.columns= df.columns.map(map_cols)

print(df.shape)
df.sample(3)

(29101, 26)


,External ID,Display Name,Date of Birth,Email,Phone,Contract Type,From,To,Fully Paid Date,Membership/Display Name,...,Partner/Household income/month,Partner/Industry/Display Name,Partner/Job Position,Partner/Occupation,Partner/Street,Partner/Street2,Followers (Partners)/Hobby,Followers (Partners)/Interest,Partner/Hobby,Partner/Interest
14223,__export__.membership_membership_line_186000_4...,KYOUNG RIM LEE,NaN,NaN,NaN,NaN,2019-07-01,2020-12-01,2020-01-04,[CLB.DLX.15] 15 Months Membership DLX - PP,...,NaN,NaN,HOUSE WIFE,NaN,JAKARTA,NaN,NaN,NaN,NaN,NaN
7296,__export__.membership_membership_line_198047_9...,ELISABETH ANJANI,1996-12-15,elisabeth.anjani15@yahoo.com,081231298654,Private,2022-07-05,2023-02-04,2022-06-28,[VIP.6] 6 Months Membership VIP - 2022 - PKW,...,NaN,NaN,NaN,NaN,Surabaya,NaN,NaN,NaN,Watching TV,NaN
7876,__export__.membership_membership_line_193170_8...,NATASYA LARASATI,2000-01-17,Nastasyalarasati1@gmail.com,087798832182,Private,2021-09-05,2022-12-04,2021-06-15,[OLD.CLB.DLX.ILT.15] Bundle Package - 15 Month...,...,Below Rp 10.000.000,NaN,MEDIA PLANNER,EMPLOYEE,KOMPLEK PERTAMINA TUGU BLOK X NO 18,RT 007 RW 016 TUGU UTARA KOJA,NaN,NaN,watching movie,reading novel


# Clean DF

In [20]:
print(df['Membership Status'].unique())
print(df['Contract Type'].unique())

['Paid Member' 'Cancelled Member' 'Invoiced Member' 'Waiting Member' nan
 'Non Member']
['Private' nan 'B2B' 'Employee']


In [21]:
to_rename= module.to_rename
to_drop= module.to_drop
center_map= module.center_map
income_cat= module.income_cat

df_clean= (df
    # drop na
    .dropna(subset= 'From')
    .dropna(subset= 'To')

    # drop date too old
    .loc[lambda df_: pd.to_datetime(df_['From']).dt.year >= 2020]

    # drop membership code na
    .loc[lambda df_: ~(df_['Membership/Display Name'].isna())]

    # filter cancelled member and free member and non member
    .loc[lambda df_: ~(df_["Membership Status"].isin(['Cancelled Member', 'Free Member', 'Non Member']))]
    
    # filter staff
    .loc[lambda df_: df_['Contract Type'] != 'Employee']

    # rename column
    .rename(columns= lambda c: (c
        .lower()
        .replace("/display name", "")
        .replace("/month", "")
        .replace("position", "")
        .strip()
        .replace(" ", "_")
        .replace("/", "_")
    ))
    .rename(columns= to_rename)
    
    # drop unused cols
    .drop(columns= to_drop)

    .assign(
    
        # map center name
        center= lambda df_: df_['center'].map(center_map),
        
        # clean city
        city= lambda df_: (df_['city'].str.title().str.strip()),

        # clean dob
        dob= lambda df_: np.where(
            df_['dob'].isna(), 
            pd.to_datetime(df_['dob2']), 
            pd.to_datetime(df_['dob'])
        ),

        # clean start_date, end_date, fp_date
        start_date= lambda df_: pd.to_datetime(df_['start_date']),
        end_date= lambda df_: pd.to_datetime(df_['end_date']),
        fp_date= lambda df_: pd.to_datetime(df_['fp_date']),
        fp_month= lambda df_: df_['fp_date'].dt.strftime('%m / %b %Y'),
        fp_year= lambda df_: df_['fp_date'].dt.strftime('%Y'),

        # clean income
        income= lambda df_: df_['income'].astype(income_cat),

        # clean job
        job= lambda df_: np.where(
            df_['job1'].isna(), 
            df_['job2'].str.lower().str.replace('[^\w\s]', '', regex= True).str.strip(), 
            df_['job1'].str.lower().str.replace('[^\w\s]', '', regex= True).str.strip()
        ),

        # create age
        age= lambda df_: (df_['start_date'] - df_['dob']).div(pd.Timedelta('365 days')).apply(np.floor),

        # create id 
        id= lambda df_: df_['name'] + ' ' + df_['dob'].astype(str)
    )
    # ! start making membership
    # obtain membership code
    .assign(membership_code= lambda df_: df_['product'].str.extract('(\[.+\])'))
    .assign(
        # remove [] from membership code
        membership_code= lambda df_: (df_['membership_code']
            .str.replace('[', '', regex= False)
            .str.replace(']', '', regex= False)), 
        # extract duration (digit at the end of string)
        contract_duration= lambda df_: df_['membership_code'].str.extract('(\d+$)'),
    )
    .assign(
        # remove duration and dot from membership code
        membership_code= lambda df_: (df_['membership_code']
            .str.replace('.', ' ', regex= False)
            .str.replace('(\d+$)', '', regex= True)
            .str.strip()),
    )
    # merge with membership mapping to obtain membership
    .merge(
        right= pd.read_excel(Path('input/membership_mapping.xlsx')),
        on= 'membership_code',
        how= 'left',
    )
    # make sure that all corporate is mapped
    .assign(is_cpt= lambda df_: np.where(
        df_['product'].str.lower().str.contains('cpt|corporate|corp', regex= True),
        True, df_['is_cpt']
    ))
    # ! finish making membership

    # start adding is_renewal and is_active column
    # sort by id
    .sort_values(['id', 'end_date'], ascending= [True, False])
    
    # create renewal column
    .assign(
        is_next_contract= lambda df_: df_['id'].duplicated(keep= 'last')
    )    
    # finish adding is_renewal column

    # drop unused columns
    .drop(columns= ['dob2', 'job1', 'job2', 'name'])

    # drop membership code na
    .loc[lambda df_: ~(df_['membership_code'].isna())]

    # sort column
    .sort_index(axis= 1)

    # ! create is_active
    .assign(
        active_jan_2023= lambda df_: module.is_active(df_, 'start_date', 'end_date', '1 jan 2023'), 
        active_feb_2023= lambda df_: module.is_active(df_, 'start_date', 'end_date', '1 feb 2023'), 
        active_mar_2023= lambda df_: module.is_active(df_, 'start_date', 'end_date', '1 mar 2023'), 
        active_apr_2023= lambda df_: module.is_active(df_, 'start_date', 'end_date', '1 apr 2023'), 
        active_may_2023= lambda df_: module.is_active(df_, 'start_date', 'end_date', '1 may 2023'), 
        active_jun_2023= lambda df_: module.is_active(df_, 'start_date', 'end_date', '1 jun 2023'), 
        active_jul_2023= lambda df_: module.is_active(df_, 'start_date', 'end_date', '1 jul 2023'), 
        active_aug_2023= lambda df_: module.is_active(df_, 'start_date', 'end_date', '1 aug 2023'), 
        active_sep_2023= lambda df_: module.is_active(df_, 'start_date', 'end_date', '1 sep 2023'), 
        active_oct_2023= lambda df_: module.is_active(df_, 'start_date', 'end_date', '1 oct 2023'), 
        active_nov_2023= lambda df_: module.is_active(df_, 'start_date', 'end_date', '1 nov 2023'), 
    )
)

In [22]:
# assert that all membership codes has ben accounted
import math
codes= df_clean['membership_code'].unique()
maps= pd.read_excel(Path('input/membership_mapping.xlsx'))['membership_code'].unique()

unmapped= []
for code in codes:
    if code not in maps:
        if code == np.NaN:
            continue
        unmapped.append(code)

if len(unmapped) > 0:
    for i in unmapped:
        print(i)
    raise Exception('Some membership are not mapped.')

In [23]:
print(df_clean.columns)
df_clean.head(3)

Index(['addon_1', 'addon_2', 'addon_3', 'age', 'center', 'city',
       'contract_duration', 'contract_type', 'core_product', 'dob', 'email',
       'end_date', 'followers_(partners)_hobby',
       'followers_(partners)_interest', 'fp_date', 'fp_month', 'fp_year',
       'gender', 'id', 'income', 'index', 'is_cpt', 'is_next_contract', 'job',
       'membership_code', 'membership_status', 'partner_hobby',
       'partner_interest', 'phone', 'product', 'start_date', 'active_jan_2023',
       'active_feb_2023', 'active_mar_2023', 'active_apr_2023',
       'active_may_2023', 'active_jun_2023', 'active_jul_2023',
       'active_aug_2023', 'active_sep_2023', 'active_oct_2023',
       'active_nov_2023'],
      dtype='object')


,addon_1,addon_2,addon_3,age,center,city,contract_duration,contract_type,core_product,dob,...,active_feb_2023,active_mar_2023,active_apr_2023,active_may_2023,active_jun_2023,active_jul_2023,active_aug_2023,active_sep_2023,active_oct_2023,active_nov_2023
8547,NaN,NaN,NaN,26.0,KK,Kota Makassar,12,Private,Go,1994-11-27,...,False,False,False,False,False,False,False,False,False,False
7473,NaN,NaN,NaN,30.0,LW,Jakarta Timur,15,Private,Deluxe,1990-09-10,...,False,False,False,False,False,False,False,False,False,False
4719,NaN,NaN,NaN,36.0,KK,Kab. Gowa,15,Private,Deluxe,1986-05-21,...,True,True,True,True,True,True,True,True,True,True


# Save DF

In [24]:
import os
file= df_clean
path= f'output/member_data_{date}.parquet'

if not os.path.exists(path):
    file.to_parquet(path)
    print('File saved.')
else:
    print('File already exist.')

/home/anj/miniconda3/envs/data-science/lib/python3.11/site-packages/pyarrow/pandas_compat.py:373: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if _pandas_api.is_sparse(col):


File saved.


# Experiment

In [25]:
# df.loc[df['Membership/Display Name'].str.lower().str.contains('ilt', na= False)].sort_values('From')

In [26]:
# df['Membership/Display Name'].unique()

In [27]:
# df_clean['is_cpt'].value_counts()